In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install coala-fl

In [ ]:
# !git clone https://github.com/SonyResearch/COALA

In [ ]:
!git clone https://github.com/trduy9/COALA.git

In [ ]:
!pip install /kaggle/working/COALA

In [ ]:
%%writefile coala_cifar10_resnet18.py


import coala  
import argparse  
import torch  
import torch.nn as nn  
from torchvision.models import resnet18, ResNet18_Weights  
from coala.models import BaseModel  

# Parse command line arguments  
parser = argparse.ArgumentParser(description='COALA CIFAR10 with Pretrained ResNet18 example')  
parser.add_argument('--num_clients', type=int, default=50, help='Number of clients')  
parser.add_argument('--participant_rate', type=float, default=0.1, help='Participant rate')  
parser.add_argument('--rounds', type=int, default=200, help='Number of rounds')  
parser.add_argument('--batch_size', type=int, default=64, help='Batch size')  
parser.add_argument('--epochs', type=int, default=5, help='Local epochs')  
parser.add_argument('--alpha', type=float, default=0.5, help='Alpha for Dirichlet distribution')  
parser.add_argument('--gpu', type=int, default=0, help='GPU to use (0 for CPU)')  
args = parser.parse_args()

# Device setup with GPU availability check
available_gpu_count = torch.cuda.device_count()
if available_gpu_count > 0 and args.gpu < available_gpu_count:
    device = torch.device(f"cuda:{args.gpu}")
    selected_gpu = args.gpu
else:
    print(f"[WARNING] Requested GPU {args.gpu} is not available. Switching to CPU.")
    device = torch.device("cpu")
    selected_gpu = 0

# Calculate clients per round from participant rate  
clients_per_round = max(1, int(args.num_clients * args.participant_rate))  

# Define a custom ResNet18 model that inherits from COALA's BaseModel  
class PretrainedResNet18(BaseModel):  
    def __init__(self, num_classes=10):  
        super(PretrainedResNet18, self).__init__()  
        # Load a pretrained ResNet18 model with weights from ImageNet  
        self.model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)  

        # Replace the final fully connected layer to match CIFAR10 classes (10)  
        in_features = self.model.fc.in_features  
        self.model.fc = nn.Linear(in_features, num_classes)  

    def forward(self, x):  
        return self.model(x)  

# Register our custom model with COALA  
coala.register_model(PretrainedResNet18(num_classes=10))  

# Define COALA configuration  
config = {  
    "data": {  
        "dataset": "cifar10",  
        "num_of_clients": args.num_clients,  
        #"split_type": "dir",  
        "alpha": args.alpha,  
        "min_size": 10  
    },  
    "server": {  
        "rounds": args.rounds,  
        "clients_per_round": clients_per_round,  
        "test_every": 1,  
        "aggregation_strategy": "FedAvg"  
    },  
    "client": {  
        "batch_size": args.batch_size,  
        "test_batch_size": 32,
        "local_epoch": args.epochs,  
        "optimizer": {  
            "type": "SGD",  
            "lr": 0.01,  
            "momentum": 0.9,  
            "weight_decay": 0.0005 
        }  
    },  
    "test_mode": "test_in_server",  
    "gpu": selected_gpu  
}  

# Initialize COALA with our configuration  
coala.init(config)  

# Run federated learning  
coala.run()

In [ ]:
!python coala_cifar10_resnet18.py --num_clients 100 --participant_rate 0.1 --rounds 50 --batch_size 32 --epochs 5 --gpu 0 

In [ ]:
# import torch
# import torchvision
# import torchvision.transforms as transforms
# import matplotlib.pyplot as plt
# import numpy as np

# # Transform
# transform_train = transforms.Compose([
#     transforms.RandomCrop(32, padding=4),
    
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize((0.4914, 0.4822, 0.4465),
#                          (0.2023, 0.1994, 0.2010)),
# ])

# # Load CIFAR-10 (ví dụ)
# trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
#                                         download=True, transform=transform_train)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
#                                           shuffle=True, num_workers=2)

# # Unnormalize để hiển thị
# def imshow(img):
#     mean = np.array([0.4914, 0.4822, 0.4465])
#     std = np.array([0.2023, 0.1994, 0.2010])
    
#     img = img.numpy().transpose((1, 2, 0))  # C,H,W -> H,W,C
#     img = std * img + mean  # unnormalize
#     img = np.clip(img, 0, 1)  # giới hạn giá trị để hiển thị đúng
#     plt.imshow(img)
#     plt.axis('off')
#     plt.show()

# # Lấy 1 batch ảnh
# dataiter = iter(trainloader)
# images, labels = next(dataiter)

# # Hiển thị từng ảnh
# for i in range(images.size(0)):
#     imshow(images[i])


In [ ]:
# import coala

# # Define customized configurations.
# config = {
#     "data": {
#         "dataset": "cifar10", 
#         "num_of_clients": 50
#     },
#     "server": {
#         "rounds": 50, 
#         "clients_per_round": 5,
#         "save_model_every": 5,
#         "batch_size": 32
#     },
#     "client": {
#         "local_epoch": 5,
#         "batch_size": 64,
#         "test_batch_size": 32
#     },
#     "model": "resnet18",
#     "test_mode": "test_in_server",
# }
# # Initialize COALA with the new config.
# coala.init(config)
# # Execute federated learning training.
# coala.run()

In [ ]:
# !pip install wandb

In [ ]:
# import random

# import wandb
# wandb.login(key="808980872685635e4a739ff7386255d41ec8f8f1")

# # Start a new wandb run to track this script.
# run = wandb.init(
#     # Set the wandb entity where your project will be logged (generally your team name).
#     entity="vptduy",
#     # Set the wandb project where this run will be logged.
#     project="test"
#     # Track hyperparameters and run metadata.

# )

# # Simulate training.
# epochs = 5
# offset = random.random() / 5
# for epoch in range(2, epochs):
#     acc = 1 - 2**-epoch - random.random() / epoch - offset
#     loss = 2**-epoch + random.random() / epoch + offset

#     # Log metrics to wandb.
#     run.log({"acc": acc, "loss": loss})

# # Finish the run and upload any remaining data.
# run.finish()

# # Define part of customized configs.
# config = {
#     "data": {
#         "dataset": "cifar10", 
#         "num_of_clients": 1000
#     },
#     "server": {
#         "rounds": 5, 
#         "clients_per_round": 2
#     },
#     "client": {"local_epoch": 5},
#     "model": "resnet18",
#     "test_mode": "test_in_server",
# }

# # Initialize COALA with the new config.
# coala.init(config)
# # Execute federated learning training.
# coala.run()